In [15]:
!which python

/Users/alanwright/Documents/GitHub/Python_Learning/DeepLearningAi/venv/bin/python


## OpenAI intro

Write a haiku about AI

In [28]:
from openai import OpenAI
import os
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

# Get API key from environment
client = OpenAI(api_key = os.getenv('OPENAI_API_KEY'))

# Use chat endpoint, complete a message, create a new chat
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": "write a haiku about ai"}
  ]
)

print(completion.choices[0].message);

ChatCompletionMessage(content='Artificial mind,\nProgrammed to learn and evolve,\nFuture now at hand.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)


#### helper function
Throughout this course, we will use OpenAI's `gpt-3.5-turbo` model and the [chat completions endpoint](https://platform.openai.com/docs/guides/chat). 

This helper function will make it easier to use prompts and look at the generated outputs.  

In [17]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

## Prompting Principles
- **Principle 1: Write clear and specific instructions**
- **Principle 2: Give the model time to “think”**

### Tactics

#### Tactic 1: Use delimiters to clearly indicate distinct parts of the input
- Delimiters can be anything like: ```, """, < >, `<tag> </tag>`, `:`

In [18]:
text = f"""
You should express what you want a model to do by providing instructions that are as clear and  specific as you can possibly make them. This will guide the model towards the desired output, 
and reduce the chances of receiving irrelevant or incorrect responses. Don't confuse writing a clear prompt with writing a short prompt. In many cases, longer prompts provide more clarity and context for the model, which can lead to more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks into a single sentence.
```{text}```
"""
response = get_completion(prompt)
print(response)

It is important to provide clear and specific instructions to guide a model towards the desired output and reduce the chances of irrelevant or incorrect responses, even if it means writing a longer prompt for more clarity and context.


#### Tactic 2: Ask for a structured output
- JSON, HTML

In [19]:
prompt = f"""
Generate a list of three made-up book titles along
with their authors and genres. 
Provide them in JSON format with the following keys: 
book_id, title, author, genre.
"""
response = get_completion(prompt)
print(response)

[
    {
        "book_id": 1,
        "title": "The Midnight Garden",
        "author": "Elena Rivers",
        "genre": "Fantasy"
    },
    {
        "book_id": 2,
        "title": "Echoes of the Past",
        "author": "Nathan Black",
        "genre": "Mystery"
    },
    {
        "book_id": 3,
        "title": "Whispers in the Wind",
        "author": "Samantha Reed",
        "genre": "Romance"
    }
]


#### Tactic 3: Ask the model to check whether conditions are satisfied

In [20]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some 
water boiling. While that's happening, 
grab a cup and put a tea bag in it. Once the water is 
hot enough, just pour it over the tea bag. 
Let it sit for a bit so the tea can steep. After a 
few minutes, take out the tea bag. If you 
like, you can add some sugar or milk to taste. 
And that's it! You've got yourself a delicious 
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, 
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:

Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Once the water is hot enough, pour it over the tea bag.
Step 4 - Let it sit for a bit so the tea can steep.
Step 5 - After a few minutes, take out the tea bag.
Step 6 - Add some sugar or milk to taste.
Step 7 - Enjoy your delicious cup of tea.


In [21]:
text_2 = f"""
The sun is shining brightly today, and the birds are 
singing. It's a beautiful day to go for a 
walk in the park. The flowers are blooming, and the 
trees are swaying gently in the breeze. People 
are out and about, enjoying the lovely weather. 
Some are having picnics, while others are playing 
games or simply relaxing on the grass. It's a 
perfect day to spend time outdoors and appreciate the 
beauty of nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, 
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

Completion for Text 2:
No steps provided.


#### Tactic 4: "Few-shot" prompting

In [22]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest 
valley flows from a modest spring; the 
grandest symphony originates from a single note; 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

<grandparent>: Resilience is like a tree that bends in the storm but does not break. It is the ability to bounce back from adversity, to persevere in the face of challenges, and to find strength in difficult times. Just as a tree grows stronger with each passing storm, so too can we grow stronger through our trials and tribulations.


### Principle 2: Give the model time to “think” 

#### Tactic 1: Specify the steps required to complete a task

In [23]:
text = f"""
In a charming village, siblings Jack and Jill set out on 
a quest to fetch water from a hilltop 
well. As they climbed, singing joyfully, misfortune 
struck—Jack tripped on a stone and tumbled 
down the hill, with Jill following suit. 
Though slightly battered, the pair returned home to 
comforting embraces. Despite the mishap, 
their adventurous spirits remained undimmed, and they 
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple 
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following 
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
1 - Jack and Jill go on a quest to fetch water, but encounter misfortune along the way, yet remain adventurous.

2 - Jack et Jill partent en quête d'eau, mais rencontrent des malheurs en chemin, tout en restant aventureux.

3 - Jack, Jill

4 - 
{
  "french_summary": "Jack et Jill partent en quête d'eau, mais rencontrent des malheurs en chemin, tout en restant aventureux.",
  "num_names": 2
}


#### Ask for output in a specified format

In [24]:
prompt_2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the 
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)


Completion for prompt 2:
Summary: Jack and Jill, two siblings, go on a quest to fetch water from a hilltop well but encounter misfortune along the way.

Translation: Jack et Jill, deux frères et sœurs, partent en quête d'eau d'un puits au sommet d'une colline mais rencontrent des malheurs en chemin.

Names: Jack, Jill

Output JSON: 
{
  "french_summary": "Jack et Jill, deux frères et sœurs, partent en quête d'eau d'un puits au sommet d'une colline mais rencontrent des malheurs en chemin.",
  "num_names": 2
}


#### Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion

In [25]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need 
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost 
me a flat $100k per year, and an additional $10 / square 
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

The student's solution is incorrect. The total cost should be calculated as follows:
Total cost = Land cost + Solar panel cost + Maintenance cost
Total cost = 100x + 250x + (100,000 + 10x)
Total cost = 350x + 100,000

Therefore, the correct total cost for the first year of operations as a function of the number of square feet is 350x + 100,000.


#### Note that the student's solution is actually not correct.
#### We can fix this by instructing the model to work out its own solution first.

In [26]:
prompt = f"""
Your task is to determine if the student's solution 
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem including the final total. 
- Then compare your solution to the student's solution 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution 
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help 
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost 
me a flat $100k per year, and an additional $10 / square 
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

To calculate the total cost for the first year of operations, we need to consider the land cost, solar panel cost, and maintenance cost.

Let x be the size of the installation in square feet.

1. Land cost: $100 * x
2. Solar panel cost: $250 * x
3. Maintenance cost: $100,000 + $10 * x

Total cost = Land cost + Solar panel cost + Maintenance cost
Total cost = $100x + $250x + $100,000 + $10x
Total cost = $360x + $100,000

Therefore, the total cost for the first year of operations as a function of the number of square feet is 360x + 100,000.

Is the student's solution the same as the actual solution just calculated:
```
No
```
Student grade:
```
Incorrect
```


## Model Limitations: Hallucinations
- Boie is a real company, the product name is not real.

In [27]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie
"""
response = get_completion(prompt)
print(response)

The AeroGlide UltraSlim Smart Toothbrush by Boie is a high-tech toothbrush designed to provide a superior cleaning experience. It features a sleek and slim design that makes it easy to hold and maneuver in your mouth. The toothbrush uses sonic vibration technology to effectively remove plaque and bacteria from your teeth and gums.

One of the standout features of the AeroGlide UltraSlim Smart Toothbrush is its smart capabilities. It connects to a mobile app via Bluetooth, allowing you to track your brushing habits and receive personalized recommendations for improving your oral hygiene routine. The app also provides real-time feedback on your brushing technique, helping you to achieve a more thorough clean.

The toothbrush is made from durable and hygienic silicone bristles that are gentle on your teeth and gums. The bristles are also antimicrobial, preventing the growth of bacteria and mold on the brush. Additionally, the AeroGlide UltraSlim Smart Toothbrush is rechargeable, making it